In [4]:
import pandas as pd
import numpy as np

CONEXION Y DESCARGA DE DATOS DE LA BASE DE DATOS

In [14]:
import pandas as pd
import sqlite3

In [22]:
def get_all():
    connection = sqlite3.connect('data/users_thebridge.db')
    cursor = connection.cursor()
    data = "SELECT * FROM USERS" #query
    result = cursor.execute(data).fetchall()
    names = [description[0] for description in cursor.description]
    connection.close()
    return pd.DataFrame(result,columns=names)

In [23]:
users = get_all()

In [24]:
users

,Date,Users,day,month,year
0,2020-01-01,13,1,1,2020
1,2020-01-02,17,2,1,2020
2,2020-01-03,13,3,1,2020
3,2020-01-04,4,4,1,2020
4,2020-01-05,10,5,1,2020
...,...,...,...,...,...
846,2022-04-26,63,26,4,2022
847,2022-04-27,64,27,4,2022
848,2022-04-28,46,28,4,2022
849,2022-04-29,29,29,4,2022


REENTRENAMIENTO DEL MODELO

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error
import pickle
from pmdarima.arima import auto_arima



In [63]:
def retrain (users_retrain):
       
    model = pickle.load(open('data/model.pkl','rb'))
          
    users_retrain['Users'] = np.log(users_retrain['Users']+1)
    
    # train = users['Users'][:821]
    # test = users['Users'][821:]   
    
    porcentaje_test = int(len(users_retrain)*0.80)
    train = users_retrain['Users'][: porcentaje_test]
    test = users_retrain['Users'][ porcentaje_test:]
    
    print (train.shape)
    print (test.shape)
    print (porcentaje_test)
    
    prediciones = model.predict(len(test))
    # print(prediciones.shape)
    # print (test.shape)
    nuevo_mape =  mean_absolute_percentage_error(test.values, prediciones)
    print (nuevo_mape)
    if nuevo_mape > 0.20:
        
        model = auto_arima(
        train,
        start_p=1,
        start_q=1,
        max_d=3,
        max_p=5,
        max_q=5,
        stationary=False, 
        random_state=42,
        trace=True,
        n_fits = 50
        )
      
        # model_retrain = Pipeline (steps = [                     
        #                 'auto_arima', auto_arima(train)])  


        # params = {
        #         'auto_arima__start_p' : range(1,5),
        #         'auto_arima__start_q': range(1,5),
        #         'auto_arima__max_d': range(1,5),
        #         'auto_arima__max_p': range(1,5),
        #         'auto_arima__max_q': range(1,5),
        #         'auto_arima__stationary': False
        #        }

    
   
        nuevo_mape_retrain =  mean_absolute_percentage_error(test.values, prediciones)
        
        return nuevo_mape_retrain

    else:
        return nuevo_mape


    #pickle.dump(model_retrain, open('model/ad_model.pkl', 'wb'))


In [64]:
resultados = retrain (users)



(680,)
(171,)
680
31.50086725455237
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-4647.001, Time=0.53 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-4412.591, Time=0.13 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-4649.135, Time=0.27 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-4581.169, Time=0.35 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-948.650, Time=0.07 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=-4647.577, Time=0.23 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-4648.597, Time=0.63 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.06 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 2.290 seconds


In [65]:
print(resultados)

31.50086725455237
